In [1]:
# Using PyTorch for NN
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
import pandas as pd
import os
import pathlib
import numpy as np

In [3]:
lda = pd.read_csv('lda_eprec6.csv')
pbe0 = pd.read_csv('pbe0_eprec6.csv')
# go through input files in geometry folder and parse geometry block into dictonary mols
#
# get relative path assuming geometries are in separate folder in the same directory
path = os.path.abspath(os.getcwd())
geometries_path = os.path.join(path, 'geometries')
mols = {}
# if relative path doesn't work use absolute path instead
#for filename in os.scandir("/home/hannes/Documents/SBU/AMS561/ML/geometries"):
for filename in os.scandir(geometries_path):
    # parse name of molecule (filename) for key
    # does not distinguish files - make sure ONLY mdns files are in director
    # e.g. no swap files from editors
    name = str(filename).split()[1]
    name = name.replace('.mdns', '')
    name = name.replace("'", "")
    name = name.replace(">", "")
    if filename.is_file():
        f = open(filename, "r")
        # list of lists containing [letter, x, y, z]
        mol = []
        # the format is
        # geometry
        # units angstrom
        #  H 0 0 0
        #  ...
        # end
        while True:
            line = f.readline()
            # skip the angstrom and geometry lines (not very clean but works for now)
            if "geometry" in line:
                line = f.readline()
                #if "units angstrom" in line:
                 #   f.readline()
                while True:
                    line = f.readline()
                    if line == "end":
                        break
                    mol.append(line.split())
                mols[name] = mol 
                #print(len(mol))       
            if not line:
                break

In [4]:
elements = {
    "H": 1,
    "He": 2,
    "Li": 3,
    "Be": 4,
    "B": 5,
    "C": 6,
    "N": 7,
    "O": 8,
    "F": 9,
    "Ne": 10,
    "Na": 11,
    "Mg": 12,
    "Al": 13,
    "Si": 14,
    "P": 15,
    "S": 16,
    "Cl": 17,
    "Ar": 18,
    "K": 19,
    "Ca": 20,
    "Sc": 21,
    "Ti": 22,
    "V": 23,
    "Cr": 24,
    "Mn": 25,
    "Fe": 26,
    "Co": 27,
    "Ni": 28,
    "Cu": 29,
    "Zn": 30,
    "Ga": 31,
    "Ge": 32,
    "As": 33,
    "Se": 34,
    "Br": 35,
    "Kr": 36,
    "Rb": 37,
    "Sr": 38,
    "Y": 39,
    "Zr": 40,
    "Nb": 41,
    "Mo": 42,
    "Tc": 43,
    "Ru": 44,
    "Rh": 45,
    "Pd": 46,
    "Ag": 47,
    "Cd": 48,
    "In": 49,
    "Sn": 50,
    "Sb": 51,
    "Te": 52,
    "I": 53,
    "Xe": 54,
    "Cs": 55,
    "Ba": 56,
    "La": 57,
    "Ce": 58,
    "Pr": 59,
    "Nd": 60,
    "Pm": 61,
    "Sm": 62,
    "Eu": 63,
    "Gd": 64,
    "Tb": 65,
    "Dy": 66,
    "Ho": 67,
    "Er": 68,
    "Tm": 69,
    "Yb": 70,
    "Lu": 71,
    "Hf": 72,
    "Ta": 73,
    "W": 74,
    "Re": 75,
    "Os": 76,
    "Ir": 77,
    "Pt": 78,
    "Au": 79,
    "Hg": 80
}
def distance(coord1, coord2):
    return np.linalg.norm(coord1 - coord2)
dimension=32
def get_charge(symbol):
    return elements[symbol]

def coulomb_matrix(molecule):
    num_atoms = len(molecule)
    coords = np.array([[float(atom[1]), float(atom[2]), float(atom[3])] for atom in molecule])
    charges = np.array([get_charge(atom[0]) for atom in molecule])
    
    coulomb_mat = np.zeros((dimension, dimension))
    
    for i in range(num_atoms):
        for j in range(num_atoms):
            if i == j:
                coulomb_mat[i, j] = 0.5 * charges[i] ** 2.4
            else:
                coulomb_mat[i, j] = charges[i] * charges[j] / distance(coords[i], coords[j])
    
    return coulomb_mat


In [5]:
mols_dict = {}

for item in mols.items():
    mols_dict[item[0]] = coulomb_matrix(item[1]).flatten() 
    #mols_vec.append(coulomb_matrix(item[1]).flatten())
    
y_test = pbe0['Energy'].astype(float)-lda['Energy'].astype(float)

In [6]:
mols_vec = []
energy_diff = []
for i in pbe0['Unnamed: 0']:
    mols_vec.append(mols_dict[i])

input_vector = np.array(mols_vec)

In [8]:
# x-test, y-test are 225 data points, remove first element for verification
x_verification = []
x_verification.append(input_vector[0])
y_verification = y_test[0]

x_input = input_vector[1:]
y_input = []
for i in y_test[1:]:
    y_input.append(i)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Input data
X = torch.tensor(x_input.tolist())
Y = torch.tensor(y_input)

# Split data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2,random_state=42)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Define the model
class NeuralNet(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model
input_size = len(x_input[0])
model = NeuralNet(input_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01) 
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

# Training loop
best_val_loss = float('inf')
patience = 10
counter = 0
for epoch in range(1000):
    # Forward pass
    outputs = model(torch.tensor(X_train, dtype=torch.float32))
    loss = criterion(outputs.squeeze(), Y_train)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Validation loss
    with torch.no_grad():
        val_outputs = model(torch.tensor(X_val, dtype=torch.float32))
        val_loss = criterion(val_outputs.squeeze(), Y_val)
        
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        counter = 0
    else:
        counter += 1
        if counter >= patience:
            print(f'Early stopping on epoch {epoch}.')
            break

    # Print progress
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}')

# Test
input_data = torch.tensor(np.array(x_verification))
input_data = scaler.transform(input_data) # Normalize
predicted_output = model(torch.tensor(input_data, dtype=torch.float32))
print("Predicted output:", predicted_output.item())
print("True output:", y_verification)



Epoch [10/1000], Loss: 0.5435, Val Loss: 13.7104
Epoch [20/1000], Loss: 0.6018, Val Loss: 6.0379
Early stopping on epoch 24.
Predicted output: -1.6381014585494995
True output: -1.9075102999999842


/tmp/ipykernel_37626/813123339.py:76: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  input_data = torch.tensor(x_verification)


In [10]:
from sklearn.metrics import mean_squared_error
def calculate_mse(model, X, Y):
    outputs = model(torch.tensor(X, dtype=torch.float32))
    loss = mean_squared_error(outputs.squeeze().detach().numpy(), Y)
    return loss

train_mse = calculate_mse(model, X_train, Y_train)
print("Training MSE:", train_mse)

val_mse = calculate_mse(model, X_val, Y_val)
print("Validation MSE:", val_mse)

Training MSE: 0.13434163
Validation MSE: 1.1032469
